In [7]:
#pip install google-cloud-bigquery

In [8]:

from google.cloud import bigquery
import pandas as pd
import numpy as np

In [9]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="../../client_secret.json"

In [10]:
client = bigquery.Client()

In [11]:
hn_dataset_ref = client.dataset('stackoverflow', project='bigquery-public-data')
hn_dataset_ref

DatasetReference('bigquery-public-data', 'stackoverflow')

In [12]:
type(hn_dataset_ref)

google.cloud.bigquery.dataset.DatasetReference

In [13]:
hn_dset = client.get_dataset(hn_dataset_ref)
hn_dset

Dataset(DatasetReference('bigquery-public-data', 'stackoverflow'))

In [14]:
type(hn_dset)

google.cloud.bigquery.dataset.Dataset

In [15]:
[x.table_id for x in client.list_tables(hn_dset)]

['badges',
 'comments',
 'post_history',
 'post_links',
 'posts_answers',
 'posts_moderator_nomination',
 'posts_orphaned_tag_wiki',
 'posts_privilege_wiki',
 'posts_questions',
 'posts_tag_wiki',
 'posts_tag_wiki_excerpt',
 'posts_wiki_placeholder',
 'stackoverflow_posts',
 'tags',
 'users',
 'votes']

In [16]:
hn_full = client.get_table(table='bigquery-public-data.stackoverflow.posts_questions')
hn_full

Table(TableReference(DatasetReference('bigquery-public-data', 'stackoverflow'), 'posts_questions'))

In [17]:
type(hn_full)

google.cloud.bigquery.table.Table

In [18]:
query_job = client.query(
        """
        SELECT
        *
        FROM `bigquery-public-data.stackoverflow.posts_questions`
        WHERE accepted_answer_id is not null AND creation_date > '2021-01-01'
        ORDER BY accepted_answer_id
        LIMIT 10"""
    )

results = query_job.result()  # Waits for job to complete.
    # [END bigquery_simple_app_query]

    # [START bigquery_simple_app_print]
# for row in results:
#     print(row)
results_list = [row for row in results]

In [19]:
cols = [field.name for field in results.schema]

In [20]:
post_questions = pd.DataFrame.from_records(results_list, columns=cols)
post_questions

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,65526420,How to check if values in individiual rows of ...,<p>Suppose I have the following data.table:</p...,65526457,2,2,None,2021-01-01 00:05:46.310000+00:00,2.0,2021-01-01 02:15:42.223000+00:00,NaT,None,NaN,None,6335637,None,1,2,r|data.table|rowwise,62
1,65526423,d3.js : Generating axis ticks for ordinal values,<p>I want to use ordinal scale in x-axis with ...,65526533,1,0,None,2021-01-01 00:06:09.007000+00:00,NaN,2021-01-01 16:25:58.447000+00:00,NaT,None,NaN,None,9574155,None,1,2,javascript|d3.js,48
2,65526490,Is there a C macro that replaces varied length...,<p>I want to be able to:</p>\n<pre><code>#defi...,65526541,1,1,None,2021-01-01 00:20:09.553000+00:00,0.0,2021-01-01 00:37:24.277000+00:00,NaT,None,NaN,None,12743240,None,1,2,c|macros,35
3,65526419,How can I construct my objects allocated throu...,<p>C++20 removed the <code>construct()</code> ...,65526554,2,0,None,2021-01-01 00:05:43.627000+00:00,NaN,2021-01-01 18:19:46.870000+00:00,2021-01-01 05:23:28.560000+00:00,None,65863.0,None,14694500,None,1,3,c++|std|c++20|allocator,351
4,65526523,Navigate from parent to child in react where e...,<p>In a React app with a parent and child elem...,65526577,1,1,None,2021-01-01 00:30:31.933000+00:00,NaN,2021-01-01 00:46:22.813000+00:00,2021-01-01 00:35:53.623000+00:00,None,8690857.0,None,3002584,None,1,2,reactjs|react-router|react-router-dom,117
5,65526451,How do I define a field in a subclass by stric...,<p>I'm trying to implement the UML diagram bel...,65526589,1,0,None,2021-01-01 00:11:17.787000+00:00,NaN,2021-01-01 10:22:51.790000+00:00,2021-01-01 10:22:51.790000+00:00,None,2458991.0,None,14634129,None,1,1,java|inheritance|uml,67
6,65526591,How to install Python 3.5.x on Ubuntu 18.04 LT...,<p>Simple question: How to install Python 3.5....,65526611,2,1,None,2021-01-01 00:49:45.320000+00:00,NaN,2021-01-01 09:48:41.703000+00:00,NaT,None,NaN,None,6305105,None,1,0,python|ubuntu,123
7,65526474,GDB: Displaying incorrect values in struct,<p>I'm trying to implement the <code>malloc</c...,65526637,1,0,None,2021-01-01 00:16:10.673000+00:00,NaN,2021-01-01 01:02:22.447000+00:00,2021-01-01 00:39:32.250000+00:00,None,1233251.0,None,11035194,None,1,1,struct|gdb|memory-address,34
8,65526590,Vue2 + laravel6 - Component implementation,<p>I just started using Vue2 with Laravel6 and...,65526639,1,0,None,2021-01-01 00:49:32.030000+00:00,NaN,2021-01-01 01:09:45.250000+00:00,2021-01-01 01:07:22.133000+00:00,None,11960598.0,None,14605909,None,1,0,laravel|vue.js|vuejs2|laravel-6,39
9,65526522,Mobile menu css,<p>What's the best way to\nachieve going from ...,65526667,2,0,None,2021-01-01 00:30:28.633000+00:00,NaN,2021-01-01 01:46:35.280000+00:00,NaT,None,NaN,None,6787542,None,1,0,css,39


In [21]:
# Pull only relevant columns to ML model
# Sample data limited to June 2021, will scale up for next steps
query_job = client.query(
        """
        SELECT
        id, accepted_answer_id, creation_date, tags
        FROM `bigquery-public-data.stackoverflow.posts_questions`
        WHERE accepted_answer_id is not null AND creation_date > '2021-05-01'
        ORDER BY accepted_answer_id"""
    )

results = query_job.result()  # Waits for job to complete.
    # [END bigquery_simple_app_query]

    # [START bigquery_simple_app_print]
# for row in results:
#     print(row)
results_list = [row for row in results]

In [22]:
cols = ['id', 'accepted_answer_id', 'question_creation_date', 'tags']

In [23]:
post_questions = pd.DataFrame.from_records(results_list, columns=cols)
post_questions

,id,accepted_answer_id,question_creation_date,tags
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,smt|formal-methods|satisfiability|cvc4
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,python
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3
...,...,...,...,...
210363,69060773,69060817,2021-09-05 05:31:01.447000+00:00,android-progressbar
210364,69060793,69060824,2021-09-05 05:34:58.063000+00:00,bash|virtual-machine
210365,69060850,69060862,2021-09-05 05:45:41.713000+00:00,intellij-idea
210366,69060801,69060898,2021-09-05 05:36:53.920000+00:00,python|pandas|dataframe|time-series|cumsum


In [24]:
query_job = client.query(
        """
        SELECT
        *
        FROM `bigquery-public-data.stackoverflow.posts_answers`
        WHERE creation_date > '2021-01-01'
        ORDER BY id
        LIMIT 10"""
    )

results = query_job.result()  # Waits for job to complete.
    # [END bigquery_simple_app_query]

    # [START bigquery_simple_app_print]
# for row in results:
#     print(row)
results_list = [row for row in results]

In [25]:
cols = [field.name for field in results.schema]

In [26]:
post_answers = pd.DataFrame.from_records(results_list, columns=cols)
post_answers

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,65526388,None,<p>Here is one solution. You can first do a gr...,None,None,0,None,2021-01-01 00:00:01.653000+00:00,None,2021-01-01 00:00:01.653000+00:00,NaT,None,NaN,None,4520420,65525964,2,2,None,None
1,65526389,None,<p>Echoing from the individual threads can be ...,None,None,0,None,2021-01-01 00:00:11.187000+00:00,None,2021-01-01 00:00:11.187000+00:00,NaT,None,NaN,None,342497,54217345,2,0,None,None
2,65526391,None,"<p>I have found similar code here (<a href=""ht...",None,None,5,None,2021-01-01 00:00:41.780000+00:00,None,2021-01-01 00:00:41.780000+00:00,NaT,None,NaN,None,13436156,65526354,2,0,None,None
3,65526393,None,"<p>Encountered the same error, just wanna add ...",None,None,0,None,2021-01-01 00:01:25.723000+00:00,None,2021-01-01 00:01:25.723000+00:00,NaT,None,NaN,None,12552972,48184969,2,0,None,None
4,65526394,None,<p>It's difficult to have all three types of v...,None,None,5,None,2021-01-01 00:01:33.950000+00:00,None,2021-01-01 00:01:33.950000+00:00,NaT,None,NaN,None,12602208,65525938,2,0,None,None
5,65526395,None,<p>to be able to change other user passwords y...,None,None,0,None,2021-01-01 00:01:32.977000+00:00,None,2021-01-01 00:01:32.977000+00:00,NaT,None,NaN,None,5562372,65523941,2,0,None,None
6,65526398,None,<blockquote>\n<p>Is it safe (and possible?) to...,None,None,2,None,2021-01-01 00:01:52.123000+00:00,None,2021-01-01 00:09:28.470000+00:00,2021-01-01 00:09:28.470000+00:00,None,14660.0,None,14660,65526234,2,1,None,None
7,65526403,None,<pre><code>def shuffle(deck):\n split_deck ...,None,None,0,None,2021-01-01 00:02:34.570000+00:00,None,2021-01-01 00:02:34.570000+00:00,NaT,None,NaN,None,4718350,65526315,2,1,None,None
8,65526404,None,"<p>FYI, here's a simple app that will do what ...",None,None,1,None,2021-01-01 00:02:40.090000+00:00,None,2021-01-01 00:09:09.673000+00:00,2021-01-01 00:09:09.673000+00:00,None,816620.0,None,816620,65514069,2,0,None,None
9,65526405,None,<p>First you'll want to check if there is no r...,None,None,0,None,2021-01-01 00:02:40.743000+00:00,None,2021-01-01 00:22:23.763000+00:00,2021-01-01 00:22:23.763000+00:00,None,14560865.0,None,14560865,65526331,2,2,None,None


In [27]:
# Pull only relevant columns to ML model
query_job = client.query(
        """
        SELECT
        id, creation_date
        FROM `bigquery-public-data.stackoverflow.posts_answers`
        WHERE creation_date > '2021-05-01'
        ORDER BY id"""
    )

results = query_job.result()  # Waits for job to complete.
    # [END bigquery_simple_app_query]

    # [START bigquery_simple_app_print]
# for row in results:
#     print(row)
results_list = [row for row in results]

In [28]:
cols = ['answer_id', 'answer_creation_date']

In [29]:
post_answers = pd.DataFrame.from_records(results_list, columns=cols)
post_answers

,answer_id,answer_creation_date
0,67341731,2021-05-01 00:00:43.057000+00:00
1,67341735,2021-05-01 00:02:14.853000+00:00
2,67341736,2021-05-01 00:02:18.417000+00:00
3,67341740,2021-05-01 00:02:59.713000+00:00
4,67341741,2021-05-01 00:03:03.637000+00:00
...,...,...
698795,69061022,2021-09-05 06:21:38.487000+00:00
698796,69061025,2021-09-05 06:22:13.693000+00:00
698797,69061028,2021-09-05 06:22:52.787000+00:00
698798,69061029,2021-09-05 06:23:42.780000+00:00


In [30]:
post_questions = post_questions.rename(columns={"id":"parent_id"})
post_questions

,parent_id,accepted_answer_id,question_creation_date,tags
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,smt|formal-methods|satisfiability|cvc4
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,python
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3
...,...,...,...,...
210363,69060773,69060817,2021-09-05 05:31:01.447000+00:00,android-progressbar
210364,69060793,69060824,2021-09-05 05:34:58.063000+00:00,bash|virtual-machine
210365,69060850,69060862,2021-09-05 05:45:41.713000+00:00,intellij-idea
210366,69060801,69060898,2021-09-05 05:36:53.920000+00:00,python|pandas|dataframe|time-series|cumsum


In [31]:
merged_df = pd.merge(post_questions, post_answers, left_on=  ['accepted_answer_id'],
                   right_on= ['answer_id'], 
                   how = 'left')
merged_df

,parent_id,accepted_answer_id,question_creation_date,tags,answer_id,answer_creation_date
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,67341801,2021-05-01 00:14:38.340000+00:00
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,smt|formal-methods|satisfiability|cvc4,67341857,2021-05-01 00:26:39.053000+00:00
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,67341911,2021-05-01 00:36:54.910000+00:00
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,python,67341961,2021-05-01 00:46:37.893000+00:00
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,67341974,2021-05-01 00:48:52.390000+00:00
...,...,...,...,...,...,...
210363,69060773,69060817,2021-09-05 05:31:01.447000+00:00,android-progressbar,69060817,2021-09-05 05:39:44.423000+00:00
210364,69060793,69060824,2021-09-05 05:34:58.063000+00:00,bash|virtual-machine,69060824,2021-09-05 05:41:47.300000+00:00
210365,69060850,69060862,2021-09-05 05:45:41.713000+00:00,intellij-idea,69060862,2021-09-05 05:51:12.690000+00:00
210366,69060801,69060898,2021-09-05 05:36:53.920000+00:00,python|pandas|dataframe|time-series|cumsum,69060898,2021-09-05 05:58:50.680000+00:00


In [32]:
# Compare accepted_answer_id and answer_id to verify identical
merged_df['accepted_answer_id'].equals(merged_df['answer_id'])

True

In [33]:
# Drop "answer_id" column because redundant
merged_df = merged_df.drop(columns=['answer_id'])
merged_df

,parent_id,accepted_answer_id,question_creation_date,tags,answer_creation_date
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,2021-05-01 00:14:38.340000+00:00
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,smt|formal-methods|satisfiability|cvc4,2021-05-01 00:26:39.053000+00:00
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,2021-05-01 00:36:54.910000+00:00
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,python,2021-05-01 00:46:37.893000+00:00
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,2021-05-01 00:48:52.390000+00:00
...,...,...,...,...,...
210363,69060773,69060817,2021-09-05 05:31:01.447000+00:00,android-progressbar,2021-09-05 05:39:44.423000+00:00
210364,69060793,69060824,2021-09-05 05:34:58.063000+00:00,bash|virtual-machine,2021-09-05 05:41:47.300000+00:00
210365,69060850,69060862,2021-09-05 05:45:41.713000+00:00,intellij-idea,2021-09-05 05:51:12.690000+00:00
210366,69060801,69060898,2021-09-05 05:36:53.920000+00:00,python|pandas|dataframe|time-series|cumsum,2021-09-05 05:58:50.680000+00:00


In [34]:
# Add column for day of question_creation_date [question_day]
merged_df['question_day'] = merged_df['question_creation_date'].dt.day_name()
merged_df

,parent_id,accepted_answer_id,question_creation_date,tags,answer_creation_date,question_day
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,2021-05-01 00:14:38.340000+00:00,Saturday
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,smt|formal-methods|satisfiability|cvc4,2021-05-01 00:26:39.053000+00:00,Saturday
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,2021-05-01 00:36:54.910000+00:00,Saturday
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,python,2021-05-01 00:46:37.893000+00:00,Saturday
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,2021-05-01 00:48:52.390000+00:00,Saturday
...,...,...,...,...,...,...
210363,69060773,69060817,2021-09-05 05:31:01.447000+00:00,android-progressbar,2021-09-05 05:39:44.423000+00:00,Sunday
210364,69060793,69060824,2021-09-05 05:34:58.063000+00:00,bash|virtual-machine,2021-09-05 05:41:47.300000+00:00,Sunday
210365,69060850,69060862,2021-09-05 05:45:41.713000+00:00,intellij-idea,2021-09-05 05:51:12.690000+00:00,Sunday
210366,69060801,69060898,2021-09-05 05:36:53.920000+00:00,python|pandas|dataframe|time-series|cumsum,2021-09-05 05:58:50.680000+00:00,Sunday


In [35]:
# Add column for hour value of question_creation_date [question_time]
merged_df['question_hour'] = merged_df['question_creation_date'].dt.hour
merged_df['question_hour_min'] = merged_df['question_creation_date'].dt.strftime('%H:%M')

merged_df

,parent_id,accepted_answer_id,question_creation_date,tags,answer_creation_date,question_day,question_hour,question_hour_min
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,2021-05-01 00:14:38.340000+00:00,Saturday,0,00:03
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,smt|formal-methods|satisfiability|cvc4,2021-05-01 00:26:39.053000+00:00,Saturday,0,00:17
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,2021-05-01 00:36:54.910000+00:00,Saturday,0,00:32
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,python,2021-05-01 00:46:37.893000+00:00,Saturday,0,00:41
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,2021-05-01 00:48:52.390000+00:00,Saturday,0,00:38
...,...,...,...,...,...,...,...,...
210363,69060773,69060817,2021-09-05 05:31:01.447000+00:00,android-progressbar,2021-09-05 05:39:44.423000+00:00,Sunday,5,05:31
210364,69060793,69060824,2021-09-05 05:34:58.063000+00:00,bash|virtual-machine,2021-09-05 05:41:47.300000+00:00,Sunday,5,05:34
210365,69060850,69060862,2021-09-05 05:45:41.713000+00:00,intellij-idea,2021-09-05 05:51:12.690000+00:00,Sunday,5,05:45
210366,69060801,69060898,2021-09-05 05:36:53.920000+00:00,python|pandas|dataframe|time-series|cumsum,2021-09-05 05:58:50.680000+00:00,Sunday,5,05:36


In [36]:
# Add column for duration till accepted answer (answer_creation_date - question_creation_date)
merged_df['accepted_answer_duration'] = (merged_df['answer_creation_date']-merged_df['question_creation_date'])/np.timedelta64(1,'h')
merged_df = merged_df.sort_values(by='accepted_answer_duration', ascending=False)
merged_df

,parent_id,accepted_answer_id,question_creation_date,tags,answer_creation_date,question_day,question_hour,question_hour_min,accepted_answer_duration
207957,67420325,69033254,2021-05-06 14:36:35.883000+00:00,windows|code-signing,2021-09-02 15:34:47.857000+00:00,Thursday,14,14:36,2856.969993
202268,67365624,68981014,2021-05-03 08:18:01.683000+00:00,node.js|express|oauth-2.0|discord,2021-08-30 08:27:48.440000+00:00,Monday,8,08:18,2856.162988
210268,67454378,69059067,2021-05-09 04:49:57.043000+00:00,java|spring|spring-boot|validation,2021-09-04 21:28:51.283000+00:00,Sunday,4,04:49,2848.648400
196437,67355370,68931282,2021-05-02 10:37:24.760000+00:00,c++|dll|uwp|dllimport|xbox,2021-08-26 00:54:18.303000+00:00,Sunday,10,10:37,2774.281540
198903,67385222,68952849,2021-05-04 12:57:32.417000+00:00,c++|linux|gcc|cmake|arm,2021-08-27 11:43:25.663000+00:00,Tuesday,12,12:57,2758.764791
...,...,...,...,...,...,...,...,...,...
113654,68262350,68262351,2021-07-05 21:41:04.843000+00:00,javascript|node.js|jestjs|tsc,2021-07-05 21:41:04.843000+00:00,Monday,21,21:41,0.000000
187252,68855357,68855358,2021-08-19 23:10:29.470000+00:00,firebase|google-cloud-functions,2021-08-19 23:10:29.470000+00:00,Thursday,23,23:10,0.000000
195492,68923827,68923828,2021-08-25 13:27:06.257000+00:00,questdb,2021-08-25 13:27:06.257000+00:00,Wednesday,13,13:27,0.000000
44851,67708855,67708856,2021-05-26 16:14:40.067000+00:00,opengl|3d|phong,2021-05-26 16:14:40.067000+00:00,Wednesday,16,16:14,0.000000


In [37]:
# Additional method to verify number of rows with zero response time (what's the possible explanation?)
zero_time = merged_df.groupby('accepted_answer_duration').count()
zero_time 

,parent_id,accepted_answer_id,question_creation_date,tags,answer_creation_date,question_day,question_hour,question_hour_min
accepted_answer_duration,,,,,,,,
0.000000,930,930,930,930,930,930,930,930
0.000166,1,1,1,1,1,1,1,1
0.000191,1,1,1,1,1,1,1,1
0.000194,1,1,1,1,1,1,1,1
0.003704,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
2758.764791,1,1,1,1,1,1,1,1
2774.281540,1,1,1,1,1,1,1,1
2848.648400,1,1,1,1,1,1,1,1


In [38]:
# drop data rows where duration is 0
indexNames = merged_df[ merged_df['accepted_answer_duration'] == 0.000000 ].index
merged_df.drop(indexNames, inplace=True)
merged_df

,parent_id,accepted_answer_id,question_creation_date,tags,answer_creation_date,question_day,question_hour,question_hour_min,accepted_answer_duration
207957,67420325,69033254,2021-05-06 14:36:35.883000+00:00,windows|code-signing,2021-09-02 15:34:47.857000+00:00,Thursday,14,14:36,2856.969993
202268,67365624,68981014,2021-05-03 08:18:01.683000+00:00,node.js|express|oauth-2.0|discord,2021-08-30 08:27:48.440000+00:00,Monday,8,08:18,2856.162988
210268,67454378,69059067,2021-05-09 04:49:57.043000+00:00,java|spring|spring-boot|validation,2021-09-04 21:28:51.283000+00:00,Sunday,4,04:49,2848.648400
196437,67355370,68931282,2021-05-02 10:37:24.760000+00:00,c++|dll|uwp|dllimport|xbox,2021-08-26 00:54:18.303000+00:00,Sunday,10,10:37,2774.281540
198903,67385222,68952849,2021-05-04 12:57:32.417000+00:00,c++|linux|gcc|cmake|arm,2021-08-27 11:43:25.663000+00:00,Tuesday,12,12:57,2758.764791
...,...,...,...,...,...,...,...,...,...
119495,68309670,68309676,2021-07-08 23:38:35.807000+00:00,java|totp|apache-commons-codec,2021-07-08 23:38:52.473000+00:00,Thursday,23,23:38,0.004629
205921,69012861,69012865,2021-09-01 11:27:42.277000+00:00,php|laravel|laravel-6,2021-09-01 11:27:55.613000+00:00,Wednesday,11,11:27,0.003704
171983,68730474,68730475,2021-08-10 16:30:19.920000+00:00,palantir-foundry|foundry-code-workbooks,2021-08-10 16:30:20.620000+00:00,Tuesday,16,16:30,0.000194
166581,68686714,68686715,2021-08-06 19:34:02.003000+00:00,palantir-foundry|foundry-slate,2021-08-06 19:34:02.690000+00:00,Friday,19,19:34,0.000191


In [39]:
# create a subset of merged_df for ML model
#response_duration_ML_df = merged_df['accepted_answer_id', 'question_day', 'question_hour', 'question_hour_min', 'accepted_answer_duration']
#response_duration_ML_df

practice_ML_df = merged_df[['accepted_answer_id', 'question_creation_date', 'question_day', 'question_hour', 'question_hour_min', 'accepted_answer_duration']]
practice_ML_df

,accepted_answer_id,question_creation_date,question_day,question_hour,question_hour_min,accepted_answer_duration
207957,69033254,2021-05-06 14:36:35.883000+00:00,Thursday,14,14:36,2856.969993
202268,68981014,2021-05-03 08:18:01.683000+00:00,Monday,8,08:18,2856.162988
210268,69059067,2021-05-09 04:49:57.043000+00:00,Sunday,4,04:49,2848.648400
196437,68931282,2021-05-02 10:37:24.760000+00:00,Sunday,10,10:37,2774.281540
198903,68952849,2021-05-04 12:57:32.417000+00:00,Tuesday,12,12:57,2758.764791
...,...,...,...,...,...,...
119495,68309676,2021-07-08 23:38:35.807000+00:00,Thursday,23,23:38,0.004629
205921,69012865,2021-09-01 11:27:42.277000+00:00,Wednesday,11,11:27,0.003704
171983,68730475,2021-08-10 16:30:19.920000+00:00,Tuesday,16,16:30,0.000194
166581,68686715,2021-08-06 19:34:02.003000+00:00,Friday,19,19:34,0.000191


In [40]:
practice_ML_df.dtypes

accepted_answer_id                        int64
question_creation_date      datetime64[ns, UTC]
question_day                             object
question_hour                             int64
question_hour_min                        object
accepted_answer_duration                float64
dtype: object

In [41]:
#key to convert day of week to numerical value
day_to_number={"Sunday":1,
              "Monday":2,
               "Tuesday":3,
               "Wednesday":4,
               "Thursday":5,
               "Friday":6,
               "Saturday":7
              }

In [42]:
#define transformation fuction
def day_convert(x):
    return day_to_number[x]

In [43]:
#apply function to column
practice_ML_df["question_day"]=practice_ML_df["question_day"].apply(day_convert)

C:\Users\Farah\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [44]:
practice_ML_df.head()

,accepted_answer_id,question_creation_date,question_day,question_hour,question_hour_min,accepted_answer_duration
207957,69033254,2021-05-06 14:36:35.883000+00:00,5,14,14:36,2856.969993
202268,68981014,2021-05-03 08:18:01.683000+00:00,2,8,08:18,2856.162988
210268,69059067,2021-05-09 04:49:57.043000+00:00,1,4,04:49,2848.648400
196437,68931282,2021-05-02 10:37:24.760000+00:00,1,10,10:37,2774.281540
198903,68952849,2021-05-04 12:57:32.417000+00:00,3,12,12:57,2758.764791


In [45]:
practice_ML_df.dtypes

accepted_answer_id                        int64
question_creation_date      datetime64[ns, UTC]
question_day                              int64
question_hour                             int64
question_hour_min                        object
accepted_answer_duration                float64
dtype: object

In [46]:
#adding column to parse minute of the hour to get minute of the day
practice_ML_df["question_hour_min"]=merged_df['question_creation_date'].dt.minute

C:\Users\Farah\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [47]:
practice_ML_df.head()

,accepted_answer_id,question_creation_date,question_day,question_hour,question_hour_min,accepted_answer_duration
207957,69033254,2021-05-06 14:36:35.883000+00:00,5,14,36,2856.969993
202268,68981014,2021-05-03 08:18:01.683000+00:00,2,8,18,2856.162988
210268,69059067,2021-05-09 04:49:57.043000+00:00,1,4,49,2848.648400
196437,68931282,2021-05-02 10:37:24.760000+00:00,1,10,37,2774.281540
198903,68952849,2021-05-04 12:57:32.417000+00:00,3,12,57,2758.764791


In [48]:
#adding column to show minute of the day question was asked

practice_ML_df["question_time"]=(practice_ML_df["question_hour"]*60)+(practice_ML_df["question_hour_min"])

C:\Users\Farah\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [49]:
practice_ML_df.head()

,accepted_answer_id,question_creation_date,question_day,question_hour,question_hour_min,accepted_answer_duration,question_time
207957,69033254,2021-05-06 14:36:35.883000+00:00,5,14,36,2856.969993,876
202268,68981014,2021-05-03 08:18:01.683000+00:00,2,8,18,2856.162988,498
210268,69059067,2021-05-09 04:49:57.043000+00:00,1,4,49,2848.648400,289
196437,68931282,2021-05-02 10:37:24.760000+00:00,1,10,37,2774.281540,637
198903,68952849,2021-05-04 12:57:32.417000+00:00,3,12,57,2758.764791,777


In [50]:
#consolidating df

practice_ML_df=practice_ML_df[["accepted_answer_id","question_creation_date","question_day","question_hour","question_time","accepted_answer_duration"]]

In [51]:
practice_ML_df.head()

,accepted_answer_id,question_creation_date,question_day,question_hour,question_time,accepted_answer_duration
207957,69033254,2021-05-06 14:36:35.883000+00:00,5,14,876,2856.969993
202268,68981014,2021-05-03 08:18:01.683000+00:00,2,8,498,2856.162988
210268,69059067,2021-05-09 04:49:57.043000+00:00,1,4,289,2848.648400
196437,68931282,2021-05-02 10:37:24.760000+00:00,1,10,637,2774.281540
198903,68952849,2021-05-04 12:57:32.417000+00:00,3,12,777,2758.764791


# Analysis starts here

## Scatterplot to show relationship between time/day and question duration

In [52]:
#scatterplot to show relationship between time/day and question duration

plt.figure(figsize=(20, 5))

features = ['question_hour', 'question_day']
target = practice_ML_df['accepted_answer_duration']

for i, col in enumerate(features):
    plt.subplot(1, len(features) , i+1)
    x = practice_ML_df[col]
    y = target
    plt.scatter(x, y, marker='o')
    plt.title(col)
    plt.xlabel(col)
    plt.ylabel("Accepted_Answer_Duration")

NameError: name 'plt' is not defined

## Regression Analysis using question_hour as feature

In [ ]:
#pandas.DatetimeIndex.dayofweek - dont use datetimeindex because the DT is not an index
# post_questions['creation_day'] = pd.DatetimeIndex.dayofweek(post_questions['creation_date'])

## pseudo code
X = practice_ML_df[['question_hour']]
y = practice_ML_df['accepted_answer_duration']


In [ ]:
plt.scatter(X, y)

In [ ]:
 # Create the model and fit the model to the data

from sklearn.linear_model import LinearRegression

model = LinearRegression()

X = X.values.reshape(-1, 1)

In [ ]:
model.fit(X, y)

In [ ]:
print('Weight coefficients: ', model.coef_)
print('y-axis intercept: ', model.intercept_)

In [ ]:
x_min = np.array([[X.min()]])
x_max = np.array([[X.max()]])
print(f"Min X Value: {x_min}")
print(f"Max X Value: {x_max}")

In [ ]:
y_min = model.predict(x_min)
y_max = model.predict(x_max)

In [ ]:
plt.scatter(X, y, c='blue')
plt.plot([x_min[0], x_max[0]], [y_min[0], y_max[0]], c='red')

In [ ]:
from sklearn.metrics import mean_squared_error

mse = (np.sqrt(mean_squared_error(X,y)))

In [ ]:
print('MSE is {}'.format(rmse))

There is no correlation between hour posted and answer duration, try with more continuous data

## Scatterplot to show relationship between minute of the day and approved answer response

<b>question_hour_min</b> was converted into minute of the day with 12 am (00:00) being 0 minutes (see column <b>question_time</b>)

In [ ]:
#scatterplot to show relationship between time/day and question duration 

plt.figure(figsize=(20, 5))

features = ['question_time']
target = practice_ML_df['accepted_answer_duration']

for i, col in enumerate(features):
    plt.subplot(1, len(features) , i+1)
    x = practice_ML_df[col]
    y = target
    plt.scatter(x, y, marker='o')
    plt.title(col)
    plt.xlabel(col)
    plt.ylabel("Accepted_Answer_Duration")

## Regression Analysis using question by minute of day

In [ ]:
#pandas.DatetimeIndex.dayofweek - dont use datetimeindex because the DT is not an index
# post_questions['creation_day'] = pd.DatetimeIndex.dayofweek(post_questions['creation_date'])

## pseudo code
X = practice_ML_df[['question_time']]
y = practice_ML_df['accepted_answer_duration']

In [ ]:
plt.scatter(X, y)

In [ ]:
 # Create the model and fit the model to the data

from sklearn.linear_model import LinearRegression

model = LinearRegression()

X = X.values.reshape(-1, 1)

In [ ]:
model.fit(X, y)

In [ ]:
print('Weight coefficients: ', model.coef_)
print('y-axis intercept: ', model.intercept_)

In [ ]:
x_min = np.array([[X.min()]])
x_max = np.array([[X.max()]])
print(f"Min X Value: {x_min}")
print(f"Max X Value: {x_max}")

In [ ]:
y_min = model.predict(x_min)
y_max = model.predict(x_max)

In [ ]:
plt.scatter(X, y, c='blue')
plt.plot([x_min[0], x_max[0]], [y_min[0], y_max[0]], c='red')

There is no correlation between the time of day the question is posted and the approved response duration <br><br>

<b>Next Steps:</b>
- try to see if there is a relationship between tags and answer_duration

- reframe question, use ML to determine whether a question has an approved answer or not

## Multiple Linear Regression

In [53]:
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [54]:
X = practice_ML_df[['question_time','question_day']]
y = practice_ML_df['accepted_answer_duration']
X.head(10)

,question_time,question_day
207957,876,5
202268,498,2
210268,289,1
196437,637,1
198903,777,3
199790,788,5
207061,1197,3
191756,473,1
207371,507,4
204750,885,3


In [ ]:
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size =0.2)
# print the data
X_train

In [ ]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression()

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
clf.predict(X_test)

In [ ]:
clf.score(X_test,y_test)